In [1]:
!pip3 install --upgrade pip
!pip3 install mysql-connector-python

In [3]:
from mysql.connector import pooling
from mysql.connector import Error

In [26]:
dbconfig={"database":"little_lemon_db", "user":"root", "password":"********"}
try:
    connection_pool = pooling.MySQLConnectionPool(pool_name='pool_b', pool_size=2, **dbconfig)
except Error as err:
    print('Connection ERROR:', err)

In [39]:
#Task1
insert_sql = '''
INSERT INTO little_lemon_db.Bookings (TableNo, GuestFirstName, GuestLastName, BookingSlot, EmployeeID) 
VALUES (8, 'Anees', 'Java', '18:00', 6);
'''

#Task2
insert_sql2 =  '''
INSERT INTO little_lemon_db.Bookings (TableNo, GuestFirstName, GuestLastName, BookingSlot, EmployeeID) 
VALUES (5, 'Bald','Vin','19:00:00',6);
'''

#Task3
insert_sql3 =  '''
INSERT INTO little_lemon_db.Bookings (TableNo, GuestFirstName, GuestLastName, BookingSlot, EmployeeID) 
VALUES (12, 'Jay','Kon','19:30:00',6);
'''

sql_list = [insert_sql, insert_sql2, insert_sql3]
connection, cursor = [], []

for i in range(4):#put 4 because 3rd one will get connection and will need to do one final loop
    try:
        connection.append(connection_pool.get_connection())
        if not connection[i]:
            print('ERROR: Connection is None')
            raise Error('Connection is None')
        
        if connection[i]:
            print(connection[i])
            cursor.append(connection[i].cursor())
            cursor[i].execute(sql_list[i-1 if i >=3 else i])
            connection[i].commit()
    except Error as error:
        print(error)
        if cursor[i]:
            cursor[i].close()
        if connection[i]:
            connection[i].close()
        if i > 2:
            print("Adding new connection in the pool.")
            connection=connector.connect(user="root",password="*******")
            pool.add_connection(cnx=connection)
            continue  
    finally:
        cursor[i].close()
        connection[i].close()


In [45]:
#alreadu close connection. Just want to see connection close error
for con in connection:
    try:
        con.close()
        print("Connection is returned to the pool")
    except Error as er:
        print("\nConnection can't be returned to the pool")
        print("Error message:", er.msg)


Connection can't be returned to the pool
Error message: Failed adding connection; queue is full

Connection can't be returned to the pool
Error message: Failed adding connection; queue is full

Connection can't be returned to the pool
Error message: Failed adding connection; queue is full

Connection can't be returned to the pool
Error message: Failed adding connection; queue is full


In [49]:
# Create a stored procedure named BasicSalesReport. 
try:
    connection = connection_pool.get_connection()
    cursor = connection.cursor()
    cursor.execute("DROP PROCEDURE IF EXISTS BasicSalesReport;")

    stored_procedure_query="""
    CREATE PROCEDURE BasicSalesReport()

    BEGIN
    SELECT 
    SUM(BillAmount) AS Total_Sale,
    AVG(BillAmount) AS Average_Sale,
    MIN(BillAmount) AS Min_Bill_Paid,
    MAX(BillAmount) AS Max_Bill_Paid
    FROM Orders;
    END
    """

    # Execute the query
    cursor.execute(stored_procedure_query)

    #********************************************#

    # Call the stored procedure with its name
    cursor.callproc("BasicSalesReport")

    # Retrieve records in "dataset"
    results = next(cursor.stored_results())
    results = results.fetchall()

    # Retrieve column names using list comprehension in a for loop 
    for column_id in cursor.stored_results():
        cols = [column[0] for column in column_id.description]


    print("Today's sales report:")
    for result in results:
        print("\t",cols[0],":",result[0])
        print("\t",cols[1],":",result[1])
        print("\t",cols[2],":",result[2])
        print("\t",cols[3],":",result[3])
except Error as error:
    print(Error)
    cursor.close()
    connection.close()
finally:
    cursor.close()
    connection.close()


Today's sales report:
	 Total_Sale : 243
	 Average_Sale : 48.6000
	 Min_Bill_Paid : 37
	 Max_Bill_Paid : 86


In [52]:
connection = connection_pool.get_connection()
cursor=connection.cursor(buffered=True)

sql_query="""SELECT 
Bookings.BookingSlot,
CONCAT(Bookings.GuestFirstName," ",Bookings.GuestLastName) AS Guest_Name,
Employees.Name AS Emp_Name,
Employees.Role AS Emp_Role
FROM Bookings 
INNER JOIN 
Employees ON Bookings.EmployeeID=Employees.EmployeeID
ORDER BY Bookings.BookingSlot ASC;"""
cursor.execute(sql_query)
results=cursor.fetchmany(size=3)
#print(cursor.column_names)
for result in results:
    print("\nBookingSlot",result[0])
    print("\tGuest_name:",result[1])
    print("\tAssigned to:",result[2],"[{}]".format(result[3]))
    
cursor.close()
connection.close()


BookingSlot 15:00:00
	Guest_name: Vanessa McCarthy
	Assigned to: Giorgos Dioudis [Head Chef]

BookingSlot 17:30:00
	Guest_name: Marcos Romero
	Assigned to: Fatma Kaya [Assistant Chef]

BookingSlot 18:00:00
	Guest_name: Anees Java
	Assigned to: John Millar [Receptionist]
